# World Boundaries

In [ ]:
using GeoJSON
using Plots

In [ ]:
using PolygonAlgorithms
using PolygonAlgorithms: x_coords, y_coords
import PolygonAlgorithms: bounds, contains

In [ ]:
using Revise
includet("modules/geoJSON.jl")
includet("modules/projection.jl")
includet("modules/affine.jl")
includet("modules/reproject.jl")

## Boundary Data

Source: https://www.geoboundaries.org/

In [ ]:
data_dir = "C:\\Users\\sinai\\Documents\\Projects\\Python\\Geospatial\\data"
geojson_filepath = joinpath(data_dir, "geoBoundariesCGAZ_ADM0.geojson");
shape_data = GeoJSON.read(geojson_filepath)

FeatureCollection with 218 Features

In [ ]:
bounds(shape_data)

(-180.0f0, -90.0f0, 180.0f0, 83.63339f0)

In [ ]:
features = filter_largest_geometries(shape_data; keep_top=3);

## Plotting

In [ ]:
output_dir = "images\\world_boundaries"

"images\\world_boundaries"

In [ ]:
function plot_geometry!(canvas, geometry::GeoJSON.AbstractGeometry; options...)
    throw("Feature geomtery of $type_ is not supported")
end

function plot_geometry!(canvas, geometry::GeoJSON.Polygon; options...)
    for region in geometry.coordinates
        plot!(canvas, [Shape(region)]; options...)
    end
    canvas
end

function plot_geometry!(canvas, geometry::GeoJSON.MultiPolygon; options...)
    for polygon in geometry.coordinates
        for region in polygon
            plot!(canvas, [Shape(region)]; options...)
        end
    end
    canvas
end

plot_geometry! (generic function with 3 methods)

In [ ]:
canvas = plot(aspect_ratio=:equal, size=(900, 500))
for (idx, shape) in enumerate(features)
    print("$idx, ")
    plot_geometry!(canvas, shape.geometry; label="", color=:black, fillalpha=0.3)
end
plot!(
    canvas, 
    xlims=(-181, 181), ylims=(-90, 90),
    xlabel="longitude (°)",
    ylabel="latitude (°)",
    title="Equirectangular",
    margin=5Plots.mm,
    );
#canvas # very slow on Jupyter notebooks

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [ ]:
output_path = joinpath(output_dir, "equirectangular.png")
savefig(canvas, output_path)

"C:\\Users\\sinai\\Documents\\Projects\\Julia\\Geospatial\\images\\world_boundaries\\equirectangular.png"

## Projections

In [ ]:
src_proj = WorldGeodeticSystem84()

WorldGeodeticSystem84(semi_major_axis=6.378137e6, flattening=1/298.25723)

Cylindrical Equal area
$$
\begin{align}
x &= R(\lambda - \lambda_0) \\
y &= R\sin \phi
\end{align}
$$

In [ ]:
dest_proj = CylindricalEqualArea(1.0f0, 0.0f0, 1.0f0)

CylindricalEqualArea{Float32}(radius=1.0, long0=0.0, k=1.0)

In [ ]:
xmin, ymax = dest_proj((-180.0, 90.0));
xmax, ymin = dest_proj((180.0, -90.0));

In [ ]:
canvas = plot(
    aspect_ratio=:equal, 
    xlims=(xmin, xmax), ylims=(ymin, ymax), 
    title="Cylindrical Equal Area",
    size=(1000, 400)
)
for (idx, shape) in enumerate(features)
    projected = reproject(shape, src_proj, dest_proj)
    plot_geometry!(canvas, projected.geometry; label="", color=:black, fillalpha=0.3)
end
#canvas # very slow on Jupyter notebooks

In [ ]:
output_path = joinpath(output_dir, "cylindrical_equal_area.png")
@time savefig(canvas, output_path)

  5.329727 seconds (737.26 k allocations: 516.472 MiB, 1.59% gc time, 4.00% compilation time)


"C:\\Users\\sinai\\Documents\\Projects\\Julia\\Geospatial\\images\\world_boundaries\\cylindrical_equal_area.png"

Mercator
$$
\begin{align}
x &= R(\lambda - \lambda_0) \\
y &= R\ln\left(\tan\left(\frac{\pi}{4} + \frac{\phi}{2}\right)\right)
\end{align}
$$

In [ ]:
dest_proj = Mercator(1.0, 0.0, 1.0)

Mercator{Float64}(radius=1.0, long0=0.0, k=1.0)

In [ ]:
function filter_latitudes(geometry::GeoJSON.Polygon{D, T}, latitude_min::Float64, latitude_max::Float64) where {D, T}
    coords = Vector{Vector{NTuple{D, T}}}()
    for region in geometry.coordinates
        coords_filtered = filter(x -> latitude_min <= x[2] <= latitude_max , region)
        push!(coords, coords_filtered)
    end
    GeoJSON.Polygon{D, T}(nothing, coords)
end

function filter_latitudes(geometry::GeoJSON.MultiPolygon{D, T}, latitude_min::Float64, latitude_max::Float64) where {D, T}
    coords = Vector{Vector{Vector{NTuple{D, T}}}}()
    for polygon in geometry.coordinates
        regions = Vector{Vector{NTuple{D, T}}}()
        for region in polygon
            coords_filtered = filter(x -> latitude_min <= x[2] <= latitude_max , region)
            push!(regions, coords_filtered)
        end
        push!(coords, regions)
    end
    GeoJSON.MultiPolygon{D, T}(nothing, coords)
end

filter_latitudes (generic function with 2 methods)

In [ ]:
xmin, ymin = dest_proj((-180.0, -85.0));
xmax, ymax = dest_proj((180.0, +85.0));
canvas = plot(
    aspect_ratio=:equal, 
    xlims=(xmin, xmax), ylims=(ymin, ymax), 
    title="Mercator",
    size=(700, 700),
)
for (idx, shape) in enumerate(features)
    geometry = shape.geometry
    geometry = filter_latitudes(geometry, -85.0, 85.0)
    projected = reproject(geometry, src_proj, dest_proj)
    plot_geometry!(canvas, projected; label="", color=:black, fillalpha=0.3)
end


In [ ]:
output_path = joinpath(output_dir, "mercator.png")
@time savefig(canvas, output_path)

  4.654981 seconds (54.17 k allocations: 478.798 MiB, 1.60% gc time, 0.18% compilation time)


"C:\\Users\\sinai\\Documents\\Projects\\Julia\\Geospatial\\images\\world_boundaries\\mercator.png"

Transverse Mercator
$$
\begin{align}
x &= R(\lambda - \lambda_0) \\
y &= R\ln\left(\tan\left(\frac{\pi}{4} + \frac{\phi}{2}\right)\right)
\end{align}
$$

In [ ]:
function split_upper_lower(coords::Vector{NTuple{D, T}}, ymin, ymax) where {D, T}
    has_mid = count(x -> ymin <= x[2] <= ymax, coords) >0
    merge_top = has_mid && count(x -> x[2] > ymax , coords) > 0
    merge_bottom = has_mid && count(x -> x[2] < ymin, coords) > 0
    if merge_top && merge_bottom
        new_coords = [coords]
    elseif merge_top
        coords_mid = filter(x -> ymin <= x[2], coords)
        coords_lower = filter(x -> x[2] < ymin , coords)
        new_coords = [coords_mid, coords_lower]
    elseif merge_bottom
        coords_upper = filter(x -> x[2] > ymax , coords)
        coords_mid = filter(x -> x[2] <= ymax, coords)
        new_coords = [coords_mid, coords_upper]
    else
        coords_upper = filter(x -> x[2] > ymax , coords)
        coords_mid = filter(x -> ymin <= x[2] <= ymax, coords)
        coords_lower = filter(x -> x[2] < ymin , coords)
        new_coords = [coords_upper, coords_mid, coords_lower]
    end
    new_coords
end

function split_upper_lower(geometry::GeoJSON.Polygon{D, T}, ymin, ymax) where {D, T}
    coords = Vector{Vector{Vector{NTuple{D, T}}}}()
    for region in geometry.coordinates
        regions_split = split_upper_lower(region, ymin, ymax)
        push!(coords, regions_split)
    end
    GeoJSON.MultiPolygon{D, T}(nothing, coords)
end

function split_upper_lower(geometry::GeoJSON.MultiPolygon{D, T}, ymin, ymax) where {D, T}
    coords = Vector{Vector{Vector{NTuple{D, T}}}}()
    for polygon in geometry.coordinates
        regions = Vector{Vector{NTuple{D, T}}}()
        for region in polygon
            regions_split = split_upper_lower(region, ymin, ymax)
            push!(regions, regions_split...)
        end
        push!(coords, regions)
    end
    GeoJSON.MultiPolygon{D, T}(nothing, coords)
end

split_upper_lower (generic function with 3 methods)

In [ ]:
function add_infinity!(coordinates::Vector{<:Tuple}; xmax=5.0)
    print("adding infinity ... ")
    idx = 1
    diff_max = 0.0
    for i in 2:length(coordinates)
        y_diff = abs(coordinates[i][2] - coordinates[i - 1][2])
        if y_diff > diff_max
            diff_max = y_diff
            idx = i
        end
    end
    is_increasing = coordinates[idx][2] > coordinates[idx-1][2]
    ymax = max(coordinates[idx][2], coordinates[idx - 1][2])
    ymin = min(coordinates[idx][2], coordinates[idx - 1][2])
    y1 = is_increasing ? ymin : ymax
    y2 = is_increasing ? ymax : ymin
    insert!(coordinates, idx, (xmax, y1))
    insert!(coordinates, idx + 1, (xmax, y2))
end

function maybe_add_infinity!(geometry::GeoJSON.Polygon,  projected::GeoJSON.Polygon, pole; options...)
    for (region, region_proj) in zip(geometry.coordinates, projected.coordinates)
        if contains(region, pole)
            add_infinity!(region_proj; options...)
        end
    end
end

function maybe_add_infinity!(geometry::GeoJSON.MultiPolygon, projected::GeoJSON.MultiPolygon, pole; options...)
    for (polygon, polygon_proj) in zip(geometry.coordinates, projected.coordinates)
        for (region, region_proj) in zip(polygon, polygon_proj)
            if contains(region, pole)
                add_infinity!(region_proj; options...)
            end
        end
    end
end

maybe_add_infinity! (generic function with 2 methods)

In [ ]:
dest_proj = TransverseMercator(1.0f0, 0.0f0, 1.0f0)

TransverseMercator{Float32}(radius=1.0, long0=0.0, k=1.0)

In [ ]:
canvas = plot(
    aspect_ratio=:equal, 
    ylims=(-π/2, 3π/2), 
    title="Transverse Mercator $(dest_proj.long0)°",
    size=(800, 600),
)
west_pole = (-90.0f0 + dest_proj.long0, 0.0f0)
east_pole = (90.0f0 + dest_proj.long0, 0.0f0)
xmax=5.0
for (idx, shape) in enumerate(features)
    print("$idx, ")
    geometry = shape.geometry
    projected = reproject(geometry, src_proj, dest_proj; extend=true)
    maybe_add_infinity!(geometry, projected, west_pole; xmax=-xmax)
    maybe_add_infinity!(geometry, projected, east_pole; xmax=+xmax)
    projected = split_upper_lower(projected, -1.0, 1.0)
    plot_geometry!(canvas, projected; label="", color=:black, fillalpha=0.3)
end
plot!([0, 0], [-π/2, π/2+π], c=:black, linestyle=:dash, label="");
xlimits = (-xmax, xlims(canvas)[2])
#xlimits = (xlims(canvas)[1], xmax)
#xlimits = xlims(canvas)
plot!(collect(xlimits), [π/2, π/2], c=:black, linestyle=:dash, label="", xlims=xlimits);

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [ ]:
output_path = joinpath(output_dir, "transverse_mercator_$(dest_proj.long0).png")
@time savefig(canvas, output_path)

  4.173966 seconds (67.76 k allocations: 479.331 MiB, 0.67% gc time)


"C:\\Users\\sinai\\Documents\\Projects\\Julia\\Geospatial\\images\\world_boundaries\\transverse_mercator_0.0.png"

Robinson

In [ ]:
dest_proj = Robinson(1.0, 0.0, 1.0, CubicSpline())

Robinson(radius=1.0, long0=0.0, k=1.0, interpolatorX=CubicSpline{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}([1.0 0.9986 … 0.6213 0.5722; -0.00020655138810036426 -0.0004268972237992455 … -0.010475096751419161 -0.008852829499594515; -2.168404344971009e-20 -4.406916713977622e-5 … 6.860460048657687e-5 0.0002558488498783523; -2.93794447598508e-6 2.897223799243307e-7 … 1.2482949959451696e-5 -1.7056589991890152e-5], 0.0:5.0:90.0), interpolatorY=CubicSpline{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}([0.0 0.062 … 0.9394 0.9761; 0.012400002113304593 0.012399995773390814 … 0.008475618700516885 0.0059183946569951705; -9.26442286059391e-24 -1.2679827558269159e-9 … -0.0001699264116057937 -0.0003415183970985493; -8.453218372179377e-11 4.226609186089726e-10 … -1.143946569951704e-5 2.2767893139903287e-5], 0.0:5.0:90.0))

In [ ]:
canvas = plot(
    aspect_ratio=:equal, 
    title="Robinson",
    size=(1000, 400)
)
for (idx, shape) in enumerate(features)
    print("$idx, ")
    projected = reproject(shape, src_proj, dest_proj)
    plot_geometry!(canvas, projected.geometry; label="", color=:black, fillalpha=0.3)
end

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [ ]:
num_points = 1000
boundary = [
    [(-180.0, lat) for lat in range(-90.0, 90.0; length=num_points)]...,
    [(180.0, lat) for lat in range(90.0, -90.0; length=num_points)]...,
]
projected = reproject(boundary, src_proj, dest_proj)
xmin, ymin, xmax, ymax = bounds(projected)
plot!(canvas, Shape(projected), fillalpha=0.0, label="", xlims=(xmin, xmax), ylims=(ymin, ymax));
#plot(Shape(projected))

In [ ]:
output_path = joinpath(output_dir, "robinson.png")
@time savefig(canvas, output_path)

  4.666155 seconds (51.58 k allocations: 479.346 MiB, 1.08% gc time, 0.27% compilation time)


"C:\\Users\\sinai\\Documents\\Projects\\Julia\\Geospatial\\images\\world_boundaries\\robinson.png"